In [3]:
pip install picle


   ---------------------------------------- 0/2 [pyreadline3]
   ---------------------------------------- 0/2 [pyreadline3]
   ---------------------------------------- 0/2 [pyreadline3]
   ---------------------------------------- 0/2 [pyreadline3]
   ---------------------------------------- 0/2 [pyreadline3]
   ---------------------------------------- 0/2 [pyreadline3]
   ---------------------------------------- 0/2 [pyreadline3]
   ---------------------------------------- 0/2 [pyreadline3]
   ---------------------------------------- 0/2 [pyreadline3]
   -------------------- ------------------- 1/2 [picle]
   ---------------------------------------- 2/2 [picle]

Note: you may need to restart the kernel to use updated packages.


In [318]:
import pickle
import numpy as np
import tensorflow

In [319]:
from keras.models import Sequential
from keras.layers import Dense

In [320]:
with open("train_qa.txt","rb") as fb:#unpickling
    train_data=pickle.load(fb)

In [321]:
with open("test_qa.txt","rb") as fb:#unpickling
    test_data=pickle.load(fb)

In [322]:
' '.join(train_data[0][0])

'Mary moved to the bathroom . Sandra journeyed to the bedroom .'

In [323]:
' '.join(train_data[0][1])

'Is Sandra in the hallway ?'

In [324]:
' '.join(train_data[0][2])

'n o'

In [325]:
type(test_data)

list

In [429]:
' '.join(train_data[3][0])

'Mary moved to the bathroom . Sandra journeyed to the bedroom . Mary went back to the bedroom . Daniel went back to the hallway . Sandra went to the kitchen . Daniel went back to the bathroom . Daniel picked up the football there . Daniel went to the bedroom .'

In [430]:
' '.join(train_data[3][1])

'Is Daniel in the bedroom ?'

In [431]:
' '.join(train_data[3][2])

'y e s'

In [329]:
' '.join(train_data[3][0])

'Mary moved to the bathroom . Sandra journeyed to the bedroom . Mary went back to the bedroom . Daniel went back to the hallway . Sandra went to the kitchen . Daniel went back to the bathroom . Daniel picked up the football there . Daniel went to the bedroom .'

In [330]:
' '.join(train_data[3][1])

'Is Daniel in the bedroom ?'

In [331]:
' '.join(train_data[3][2])

'y e s'

In [332]:
len(test_data)

1000

In [333]:
len(train_data)

10000

In [334]:
train_data[0]

(['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'Sandra',
  'journeyed',
  'to',
  'the',
  'bedroom',
  '.'],
 ['Is', 'Sandra', 'in', 'the', 'hallway', '?'],
 'no')

In [335]:
#create a set that holds the vocab words
vocab=set()

In [336]:
all_data=test_data+train_data

In [337]:
for story,question,answer in all_data:
    #in case you don't know what a union of sets is:
    vocab=vocab.union(set(story))
    vocab=vocab.union(set(question))

In [338]:
vocab.add('no')
vocab.add('yes')

In [339]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [340]:
vocab_len=len(vocab)+1#we add an extra space to hold a 0

In [341]:
max_story_len=max([len(data[0]) for data in all_data])

In [342]:
max_story_len

156

In [343]:
max_question_len=max([len(data[1]) for data in all_data])

In [344]:
max_question_len

6

In [345]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [346]:
#reserve a for pad_sequence
#keras is for front end and tensor for back end
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer()

vocab_size=len(tokenizer.word_index)+1
print("Voca size:",vocab_size)

Voca size: 1


In [347]:
import keras

In [348]:
!pip install keras --user

In [349]:
from keras.preprocessing.sequence import pad_sequences

In [350]:
text=['Mary moved to the bathroom Sandara hourneyed to the bedroom.']

In [351]:
tokenizer=Tokenizer(num_words=18)

In [352]:
#tokenizer.fit_on_texts(text)

In [353]:
#sequences=tokenizer.texts_to_sequences(test)

In [354]:
#integer encode sequences of words
#tokenizer=Tokenizer(filters=[])

In [355]:
#tokenizer=Tokenizer(num_words=18)


In [356]:
#tokenizer.fit_on_texts(text)

In [357]:
#sequences=tokenizer.texts_to_sequences(text)

In [358]:
#integer encode sequences of words
#tokenizer=Tokenizer(filters=[])
#tokenizer.fit_on_texts(vocab)

In [359]:
#tokenizer.word_index

In [360]:
tokenizer = Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)

vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

38


In [361]:
train_story_text=[]
train_question_text=[]
train_answer=[]
for story,question,answer in train_data:
    train_story_text.append(story)
    train_question_text.append(question)

In [362]:
train_story_seq=tokenizer.texts_to_sequences(train_story_text)

In [363]:
len(train_story_text)

10000

In [364]:
len(train_story_seq)

10000

In [365]:
import numpy as np

In [366]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def vectorize_stories(data,word_index=tokenizer.word_index, max_story_len=max_story_len,max_question_len=max_question_len):
    X = []
    Xq = []
    Y = []
    for story, query, answer in data:

        if '<unk>' not in word_index:
            word_index['<unk>'] = len(word_index) + 1

        # Story word index
        xs = [word_index.get(word.lower(), word_index['<unk>']) 
              for word in story]

        # Query word index
        xq = [word_index.get(word.lower(), word_index['<unk>']) 
              for word in query]

        # One-hot encoding for answer
        y = np.zeros(len(word_index) + 1)
        y[word_index.get(answer.lower(), word_index['<unk>'])] = 1

        X.append(xs)
        Xq.append(xq)
        Y.append(y)

    return (pad_sequences(X, maxlen=max_story_len),
            pad_sequences(Xq, maxlen=max_question_len),
            np.array(Y))

In [367]:
inputs_train,queries_train,answers_train=vectorize_stories(train_data)

In [368]:
inputs_train.shape

(10000, 156)

In [369]:
queries_train.shape

(10000, 6)

In [370]:
answers_train.shape

(10000, 39)

In [371]:
inputs_test,queries_test,answers_test=vectorize_stories(test_data)

In [372]:
inputs_test

array([[ 0,  0,  0, ..., 32,  5, 37],
       [ 0,  0,  0, ..., 32, 28, 37],
       [ 0,  0,  0, ..., 32, 28, 37],
       ...,
       [ 0,  0,  0, ..., 32, 22, 37],
       [ 0,  0,  0, ..., 32, 28, 37],
       [ 0,  0,  0, ..., 22,  4, 37]], shape=(1000, 156), dtype=int32)

In [373]:
queries_test

array([[12,  9,  8, 32, 27,  7],
       [12,  9,  8, 32, 27,  7],
       [12,  9,  8, 32, 28,  7],
       ...,
       [12,  1,  8, 32,  5,  7],
       [12, 26,  8, 32, 28,  7],
       [12,  1,  8, 32, 28,  7]], shape=(1000, 6), dtype=int32)

In [374]:
answers_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(1000, 39))

In [375]:
sum(answers_test)

array([  0.,   0.,   0.,   0.,   0.,   0., 497.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 503.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.])

In [376]:
tokenizer.word_index['yes']

6

In [377]:
tokenizer.word_index['no']

31

In [378]:
from keras.models import Sequential,Model
from keras.layers import Embedding
from keras.layers import Input,Activation,Dense,Permute,Dropout
from keras.layers import add,dot,concatenate
from keras.layers import LSTM

In [379]:
input_sequence=Input((max_story_len,))
question=Input((max_question_len,))

In [380]:
#input gets embedded to a sequence of vectors
input_encoder_m=Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,output_dim=64))
input_encoder_m.add(Dropout(0.3))
#this encoder will output:
#(samples. story maxlen, embedding dim)

In [381]:
#embed the input into a sequence of vectors of size query_maxlen
input_encoder_c=Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,output_dim=max_question_len))
input_encoder_c.add(Dropout(0.3))
#output:(samles.story_maxlen,query_maxlen)

In [382]:
question_encoder=Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,output_dim=64,input_length=max_question_len))
question_encoder.add(Dropout(0.3))

In [383]:
input_encoded_m=input_encoder_m(input_sequence)
input_encoded_c=input_encoder_c(input_sequence)
question_encoded=question_encoder(question)

In [384]:
match=dot([input_encoded_m,question_encoded],axes=(2,2))
match=Activation('softmax')(match)

In [385]:
response=add([match,input_encoded_c])
response=Permute((2,1))(response)

In [386]:
answer=concatenate([response,question_encoded])

In [387]:
answer=LSTM(32,return_sequences=False,)(answer)

In [388]:
answer

<KerasTensor shape=(None, 32), dtype=float32, sparse=False, ragged=False, name=keras_tensor_143>

In [389]:
answer=Dropout(0.5)(answer)
answer=Dense(vocab_size)(answer)

In [390]:
model=Model([input_sequence,question],answer)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [391]:
#build the final model
model=Model([input_sequence,question],answer)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [392]:
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, concatenate
from tensorflow.keras.models import Model

max_len_input = inputs_train.shape[1]
max_len_query = queries_train.shape[1]

# Story input
input1 = Input(shape=(max_len_input,))
x1 = Embedding(vocab_size, 64)(input1)
x1 = LSTM(32)(x1)

# Question input
input2 = Input(shape=(max_len_query,))
x2 = Embedding(vocab_size, 64)(input2)
x2 = LSTM(32)(x2)

# Merge both
merged = concatenate([x1, x2])

# Output layer (39 classes)
output = Dense(39, activation='softmax')(merged)

model = Model(inputs=[input1, input2], outputs=output)

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

Model: "functional_27"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_39 (InputLayer)   │ (None, 156)               │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_40 (InputLayer)   │ (None, 6)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_33 (Embedding)      │ (None, 156, 64)           │           2,432 │ input_layer_39[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_34 (Embedding)      │ (None, 6, 64)             │           2,432 │ input_layer_40[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_21 (LSTM)                │ (None, 32)                │          12,416 │ embedding_33[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_22 (LSTM)                │ (None, 32)                │          12,416 │ embedding_34[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_10 (Concatenate)  │ (None, 64)                │               0 │ lstm_21[0][0],             │
│                               │                           │                 │ lstm_22[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_10 (Dense)              │ (None, 39)                │           2,535 │ concatenate_10[0][0]       │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 32,231 (125.90 KB)

 Trainable params: 32,231 (125.90 KB)

 Non-trainable params: 0 (0.00 B)

In [393]:
vocab_size = len(tokenizer.word_index) + 1

In [394]:
history=model.fit([inputs_train,queries_train],answers_train,batch_size=32,epochs=30,validation_data=([inputs_test,queries_test],answers_test))

Epoch 1/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 28s 70ms/step - accuracy: 0.4929 - loss: 0.8799 - val_accuracy: 0.5030 - val_loss: 0.6988
Epoch 2/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 39s 65ms/step - accuracy: 0.4922 - loss: 0.6970 - val_accuracy: 0.4970 - val_loss: 0.6945
Epoch 3/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 22s 69ms/step - accuracy: 0.4938 - loss: 0.6962 - val_accuracy: 0.5030 - val_loss: 0.6951
Epoch 4/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 40s 66ms/step - accuracy: 0.4951 - loss: 0.6979 - val_accuracy: 0.4970 - val_loss: 0.6964
Epoch 5/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 39s 57ms/step - accuracy: 0.4954 - loss: 0.6967 - val_accuracy: 0.4970 - val_loss: 0.6961
Epoch 6/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 17s 54ms/step - accuracy: 0.4976 - loss: 0.6964 - val_accuracy: 0.5030 - val_loss: 0.6939
Epoch 7/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 21s 56ms/step - accuracy: 0.4997 - loss: 0.6965 - val_accuracy: 0.4970 - val_loss: 0.6933
Epoch 8/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 54ms/step - accuracy: 0.4997 - loss: 0.6963 - 

In [407]:
filename='chatbot_120_epochs.h5'
model.save(filename)

In [409]:
model.load_weights(filename)
pred_results=model.predict(([inputs_test,queries_test]))

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step


In [410]:
story=' '.join(word for word in test_data[0][0])
print(story)

Mary got the milk there . John moved to the bedroom .


In [411]:
query=' '.join(word for word in test_data[0][1])
print(query)

Is John in the kitchen ?


In [412]:
print("true test answer from data is:",test_data[0][2])

true test answer from data is: no


In [413]:
#generate prediction from model
val_max=np.argmax(pred_results[0])

for key,val in tokenizer.word_index.items():
    if val==val_max:
        k=key

print("predicted answer is:",k)
print("probability of certainty was:",pred_results[0][val_max])

predicted answer is: yes
probability of certainty was: 0.51164126


In [414]:
my_story="jhon travelled the bathroom last saturday. Sandra dropped the football in the garden."
my_story

'jhon travelled the bathroom last saturday. Sandra dropped the football in the garden.'

In [415]:
my_question="did john travelled the bathroom last friday?"

In [416]:
my_question.split()

['did', 'john', 'travelled', 'the', 'bathroom', 'last', 'friday?']

In [417]:
mydata=[(my_story,my_question,'yes')]

In [424]:
my_story,my_ques,my_ans=vectorize_stories(mydata)

In [425]:
pred_results=model.predict([my_story,my_ques])

InvalidArgumentError: Graph execution error:

Detected at node functional_27_1/embedding_33_1/GatherV2 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\Home\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "C:\Users\Home\AppData\Local\Programs\Python\Python311\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\Home\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\kernelapp.py", line 758, in start

  File "C:\Users\Home\AppData\Local\Programs\Python\Python311\Lib\site-packages\tornado\platform\asyncio.py", line 211, in start

  File "C:\Users\Home\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 608, in run_forever

  File "C:\Users\Home\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 1936, in _run_o

In [421]:

#generate prediction from model
val_max=np.argmax(pred_results[0])

for key,val in tokenizer.word_index.items():
    if val==val_max:
        k=key

print("predicted answer is:",k)
print("probability of certainty was:",pred_results[0][val_max])

predicted answer is: yes
probability of certainty was: 0.51164126
